In [ ]:
from Bio import SeqIO
import pandas as pd

### `Collects and annotates significant and suggestive results from GWAS output`

In [ ]:
#load annotations and blast output
adf = pd.read_csv('BLAST/IRGSP-1.0_representative_annotation_2020-09-09.tsv', sep = '\t')
gdf = pd.read_csv('BLAST/BoutAllgenes.csv', header = None, sep = '\t')
gdf[12] = gdf[[8,9]].min(axis = 1)
gdf[13] = gdf[[8,9]].max(axis = 1)
gdf[1] = gdf[1].apply(lambda x: int(x[-2:]))

In [ ]:
#for whole population
sig, sug = [], []

for file in os.listdir('GWAS_factory/WF2noFDR/'):
    if file[-16:] == 'GWAS.Results.csv':
        df = pd.read_csv('GWAS_factory/WF2noFDR/%s' %file)
        print('Working with %s.' %file)
        for c,p, pv in df[df['P.value'] <= 1e-5][['Chromosome', 'Position ', 'P.value']].itertuples(index = False):
            
            if pv <= 3.16e-7:
                for g,s,e in gdf[gdf[1] == c][[0, 12, 13]].itertuples(index = False):
                    if int(s) <= int(p) <= int(e):
                        sig.append((file, g, c, p, pv))
                    else:
                        sig.append((file, 'None', c, p, pv))
                        
            else:
                for g,s,e in gdf[gdf[1] == c][[0, 12, 13]].itertuples(index = False):
                    if int(s) <= int(p) <= int(e):
                        sug.append((file, g, c, p, pv))
                    else:
                        sug.append((file, 'None', c, p, pv))

In [ ]:
sigdf = pd.DataFrame(sig)
sugdf = pd.DataFrame(sug)
pd.merge(sigdf, adf, left_on= 1, right_on='Transcript_ID').sort_values(0).drop_duplicates([0,1]).to_csv('BLAST/SigAnned.csv', index = False)
pd.merge(sugdf, adf, left_on= 1, right_on='Transcript_ID').sort_values(0).drop_duplicates([0,1]).to_csv('BLAST/SugAnned.csv', index = False)

In [ ]:
#for aus subpopulation
sig, sug = [], []

for file in os.listdir('GWAS_factory/WF2ausNoFDR/'): #spot
    if file[-16:] == 'GWAS.Results.csv':
        df = pd.read_csv('GWAS_factory/WF2ausNoFDR/%s' %file) #spot
        print('Working with %s.' %file)
        for c,p, pv in df[df['P.value'] <= 1e-5][['Chromosome', 'Position ', 'P.value']].itertuples(index = False):
            
            if pv <= 3.98e-7:
                for g,s,e in gdf[gdf[1] == c][[0, 12, 13]].itertuples(index = False):
                    if int(s) <= int(p) <= int(e):
                        sig.append((file, g, c, p, pv))
                    else:
                        sig.append((file, 'None', c, p, pv))
                        
            else:
                for g,s,e in gdf[gdf[1] == c][[0, 12, 13]].itertuples(index = False):
                    if int(s) <= int(p) <= int(e):
                        sug.append((file, g, c, p, pv))
                    else:
                        sug.append((file, 'None', c, p, pv))

In [ ]:
sigdf = pd.DataFrame(sig)
sugdf = pd.DataFrame(sug)
pd.merge(sigdf, adf, left_on= 1, right_on='Transcript_ID').sort_values(0).drop_duplicates([0,1]).to_csv('BLAST/AusSigAnned.csv', index = False)
pd.merge(sugdf, adf, left_on= 1, right_on='Transcript_ID').sort_values(0).drop_duplicates([0,1]).to_csv('BLAST/AusSugAnned.csv', index = False)

In [ ]:
#for indica subpopulation
sig, sug = [], []

for file in os.listdir('GWAS_factory/WF2indNoFDR/'): #spot
    if file[-16:] == 'GWAS.Results.csv':
        df = pd.read_csv('GWAS_factory/WF2indNoFDR/%s' %file) #spot
        print('Working with %s.' %file)
        for c,p, pv in df[df['P.value'] <= 1e-5][['Chromosome', 'Position ', 'P.value']].itertuples(index = False):
            
            if pv <= 3.98e-7:
                for g,s,e in gdf[gdf[1] == c][[0, 12, 13]].itertuples(index = False):
                    if int(s) <= int(p) <= int(e):
                        sig.append((file, g, c, p, pv))
                    else:
                        sig.append((file, 'None', c, p, pv))
                        
            else:
                for g,s,e in gdf[gdf[1] == c][[0, 12, 13]].itertuples(index = False):
                    if int(s) <= int(p) <= int(e):
                        sug.append((file, g, c, p, pv))
                    else:
                        sug.append((file, 'None', c, p, pv))

In [ ]:
sigdf = pd.DataFrame(sig)
sugdf = pd.DataFrame(sug)
pd.merge(sigdf, adf, left_on= 1, right_on='Transcript_ID').sort_values(0).drop_duplicates([0,1]).to_csv('BLAST/IndSigAnned.csv', index = False)
pd.merge(sugdf, adf, left_on= 1, right_on='Transcript_ID').sort_values(0).drop_duplicates([0,1]).to_csv('BLAST/IndSugAnned.csv', index = False)

### `Gathers ranges of exons in the rice genome`

In [10]:
yd = []
for sr in SeqIO.parse('../Data/RAP-DB/IRGSP-1.0_gene_2020-06-03.fasta', 'fasta'):
    yd.append((sr.id, ' '.join(sr.description.split(' ')[1:])))

In [13]:
rg = pd.DataFrame(yd)

In [16]:
yd = []
for sr in SeqIO.parse('../Data/RAP-DB/IRGSP-1.0_cds_2020-06-03.fasta', 'fasta'):
    yd.append((sr.id, ' '.join(sr.description.split(' ')[1:])))

In [17]:
rc = pd.DataFrame(yd)

In [45]:
rc['Hypothetical'] = 0
for i,r in rc.iterrows():
    if 'hypothetical' in r[1].lower():
        rc.loc[i, 'Hypothetical'] = 1

In [60]:
for sr in SeqIO.parse('../Data/RAP-DB/IRGSP-1.0_gene_2020-06-03.fasta', 'fasta'):
    if sr.id in rc[rc['Hypothetical'] == 0][0].to_list():
        with open('Putative RAPDB genes.fasta', 'a') as f:
            f.write('>%s\n%s\n' %(sr.description, sr.seq))

In [61]:
for sr in SeqIO.parse('../Data/RAP-DB/IRGSP-1.0_cds_2020-06-03.fasta', 'fasta'):
    if sr.id in rc[rc['Hypothetical'] == 0][0].to_list():
        with open('Putative RAPDB cds.fasta', 'a') as f:
            f.write('>%s\n%s\n' %(sr.description, sr.seq))

In [83]:
df = pd.read_table('../Data/BLASTout/panvsexonsout.tsv', header = None)
df = df[df[2] == 100].reset_index(drop = True)

ndf = pd.DataFrame()
for i in range(1,13):
    c = 'chr' + str(i).zfill(2)
    tdf = df[df[1] == c]
    ndf = pd.concat([ndf,tdf], axis = 0).reset_index(drop = True)
df = ndf
del ndf
df.to_csv('../Data/BLASTout/pan vs exon 100, chronly.csv', index = False)